# Introduction


The efficient scheduling of classes in Higher Education Institutions (HEIs) is crucial for ensuring optimal resource utilization and a seamless learning experience. However, manual scheduling processes are prone to errors and often fail to account for complex interdependencies between factors such as teacher availability, classroom capacities, and course requirements. To address these challenges, this project focuses on developing an automated scheduling system using advanced computational tools. By leveraging optimization techniques and prescriptive analysis, the system aims to generate conflict-free schedules that meet all established criteria, fostering a harmonized educational environment and enhancing the overall efficiency of HEI operations.

# Main Objectives

1. The main objective is to create an automated system for generating schedules in an HEI.<br>
2. The system should optimize the allocation of resources such as physical spaces and teaching schedules to meet the needs of students and faculty.<br>
3. It should aim to avoid conflicts and unwanted overlaps in schedules.<br>
4. The solution should employ optimization techniques and prescriptive analysis to ensure the harmonization and effectiveness of schedules.<br>



# Instalations

Installed Pyomo using the pip command. Faced difficulties installing the GLPK library, only managing to install it using Anaconda. That's why I had to specify the directory in the solver.</br>
<i>pip install pyomo</I>
<i>conda install -c conda-forge glpk</i>

# Definição do problema

Durante a elaboração deste problema defini algumas regras basicas, o horario de aulas sera compreendido entre as 09:00 da manhã e as 18:00 da tarde, de segunda a sexta-feira. Simulando assim um horario de aulas do IPCA.
Com estas constrições temos: 
Não pode haver mais que 8 horas de aulas por dia;
Os alunos tem que ter pelo menos uma manhã ou uma tarde livres;



# Importing Pyomo

In [12]:
import pyomo.environ as pyo

# Constants and data definition

I've set up constants like weekends, maximum lessons per day, and days of the week. I've defined two classes with their respective subjects and student numbers. Then, I've calculated the total number of students for each class. Additionally, I've specified available hours per week for each lesson and assigned teachers to their respective lessons. I've also noted teacher unavailability and classroom capacities.


In [13]:
# Define constants
WEEKENDS = ['Saturday', 'Sunday']
MAX_LESSONS_PER_DAY = 4
MAX_DAYS_PER_WEEK = 5

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_slot = ['Morning_09_11', 'Morning_11_13', 'Afternoon_14_16', 'Afternoon_16_18']

# class1 = ['Math_1', 'Math_2', 'Science1', 'English1', 'IT', 'IT_LAB']
# class2 = ['History', 'Literature', 'Art', 'Music', 'Drama', 'Creative_Writing']
# Define classes and number of students, class 1 have 30 students class 2 have 25
class1 = {'Math_1': 30, 'Math_2': 30, 'Science1': 30, 'English1': 30, 'IT': 30, 'IT_LAB': 30} 
class2 = {'History': 25, 'Literature': 25, 'Art': 25, 'Music': 25, 'Drama': 25, 'Creative_Writing': 25}

# Define number of students for each class
num_students = {**class1, **class2}

# Define available hours per week for each lesson
lesson_hours = {'Math_1': 8, 'Math_2': 8, 'Science1': 6, 'English1': 2,
                'IT': 8, 'IT_LAB': 4, 'History': 8, 'Literature': 8,
                'Art': 4, 'Music': 2, 'Drama': 6, 'Creative_Writing': 8}

# Define teachers and their lessons
teachers = ['John', 'Isabella', 'Brandon', 'Rafael', 'Lyla', 'Smith']
teacher_lessons = {
    'John': ['Math_1', 'Math_2'],
    'Isabella': ['History'],
    'Brandon': ['Science1', 'English1', 'Literature'],
    'Rafael': ['Art', 'Music', 'Drama'],
    'Lyla': ['IT', 'IT_LAB'],
    'Smith': ['IT_LAB']
}

teacher_unavailability = {
    'Isabella': ['Monday', 'Friday'],
    'John': ['Morning_09_11', 'Morning_11_13']
}

# Define classrooms and their capacities
classrooms = ['IT_Classroom', 'Art_Classroom', 'Classroom_1', 'Classroom_2', 'Classroom_3']
# the intention here is to force a classroom to never be used
classroom_capacity = {'IT_Classroom': 30, 'Art_Classroom': 25, 'Classroom_1': 35, 'Classroom_2': 22, 'Classroom_3': 40} 


# Model Creation

I've created a Pyomo ConcreteModel to formulate my optimization problem. Then, I've defined sets for teachers, lessons, days, and time slots based on the given data. I've also included classrooms as a set.

For decision variables, I've defined classroom_assignment to represent whether a lesson is assigned to a specific classroom at a certain day and time slot. Additionally, I've defined schedule variables to indicate whether a teacher is scheduled to teach a lesson on a particular day and time slot. Both variables are binary decision variables.

In [14]:
# Define model
model = pyo.ConcreteModel()

# Sets
model.teachers = pyo.Set(initialize=teachers)
model.lessons = pyo.Set(initialize=list(class1.keys()) + list(class2.keys()))
model.days = pyo.Set(initialize=[day for day in days if day not in WEEKENDS]) # since we only have school in week days
model.time_slots = pyo.Set(initialize=time_slot)

# Update the model to include classrooms as a set
model.classrooms = pyo.Set(initialize=classrooms)

# Decision Variables: Classroom assignments
model.classroom_assignment = pyo.Var(model.lessons, model.classrooms, 
                                     model.days, model.time_slots, 
                                     domain=pyo.Binary)

# Decision Variables
model.schedule = pyo.Var(model.teachers, 
                         model.days, 
                         model.time_slots, 
                         model.lessons, 
                         domain=pyo.Binary)

# Constraints

In this code, I've defined several constraints to ensure the scheduling optimization problem adheres to specific rules and preferences.

In [32]:
# Constraint: IT and IT_LAB can only have lessons in the IT_Classroom
def it_classroom_constraint(model, lesson, classroom, day, time):
    if lesson in ['IT', 'IT_LAB']:
        if classroom != 'IT_Classroom':
            print(f"IT classroom constraint violated for lesson {lesson}, classroom {classroom}, day {day}, time {time}")
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

# Remove the existing constraint if it already exists
if hasattr(model, 'it_classroom_constraint'):
    del model.it_classroom_constraint

# Define the constraint
model.it_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                               model.days, model.time_slots, 
                                               rule=it_classroom_constraint)

In [19]:
# Constraint: Art can only have lessons in the Art_Classroom
def art_classroom_constraint_rule(model, lesson, classroom, day, time):
    if lesson == 'Art' and classroom != 'Art_Classroom':
        return model.classroom_assignment[lesson, classroom, day, time] == 0
    else:
        return pyo.Constraint.Skip

# Remove the existing constraint if it already exists
if hasattr(model, 'art_classroom_constraint'):
    del model.art_classroom_constraint

# Define the constraint
model.art_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                                model.days, model.time_slots, 
                                                rule=art_classroom_constraint_rule)



In [22]:
# Constraint: Ensure classroom capacity is not exceeded
def classroom_capacity_constraint_rule(model, classroom, day, time):
    total_students = sum(model.classroom_assignment[lesson, classroom, day, time] * num_students[lesson]
                         for lesson in model.lessons)
    return total_students <= classroom_capacity[classroom]

# Remove the existing constraint if it already exists
if hasattr(model, 'classroom_capacity_constraint'):
    del model.classroom_capacity_constraint

# Define the constraint
model.classroom_capacity_constraint = pyo.Constraint(model.classrooms, model.days, model.time_slots, 
                                                      rule=classroom_capacity_constraint_rule)


In [23]:
# Constraint: Total hours for each lesson should not exceed its availability
def availability_constraint_rule(model, lesson):
    total_hours = sum(model.schedule[teacher, day, time, lesson] 
                      for teacher in model.teachers
                      for day in model.days
                      for time in model.time_slots)
    return total_hours <= lesson_hours[lesson]

# Remove the existing constraint if it already exists
if hasattr(model, 'availability_constraint'):
    del model.availability_constraint

# Define the constraint
model.availability_constraint = pyo.Constraint(model.lessons, rule=availability_constraint_rule)


In [24]:
# Constraint: Isabella prefers not to work on Mondays and Fridays
def isabella_unavailability_constraint_rule(model, teacher, day):
    if teacher == 'Isabella' and day in teacher_unavailability['Isabella']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for time in model.time_slots
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

# Remove the existing constraint if it already exists
if hasattr(model, 'isabella_unavailability_constraint'):
    del model.isabella_unavailability_constraint

# Define the constraint
model.isabella_unavailability_constraint = pyo.Constraint(model.teachers, model.days, rule=isabella_unavailability_constraint_rule)


In [25]:
# Constraint: John can only work in the afternoon
def john_unavailability_constraint_rule(model, teacher, time):
    if teacher == 'John' and time in teacher_unavailability['John']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for day in model.days
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

# Remove the existing constraint if it already exists
if hasattr(model, 'john_unavailability_constraint'):
    del model.john_unavailability_constraint

# Define the constraint
model.john_unavailability_constraint = pyo.Constraint(model.teachers, model.time_slots, rule=john_unavailability_constraint_rule)


In [26]:
# Constraint: Isabella prefers not to work on Mondays and Fridays
def isabella_unavailability_constraint_rule(model, teacher, day):
    if teacher == 'Isabella' and day in teacher_unavailability['Isabella']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for time in model.time_slots
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

# Remove the existing constraint if it already exists
if hasattr(model, 'isabella_unavailability_constraint'):
    del model.isabella_unavailability_constraint

# Define the constraint
model.isabella_unavailability_constraint = pyo.Constraint(model.teachers, model.days, rule=isabella_unavailability_constraint_rule)


# Creating solver

Here, I'm solving the optimization model using the GLPK (GNU Linear Programming Kit) solver. I've specified the solver to be used as 'glpk' and provided the path to the GLPSOL executable file. This command instructs Pyomo to utilize the GLPK solver to find the optimal solution for the defined mathematical model. The results of the optimization process are stored in the results variable.

In [29]:

# Solve the model
# solver = pyo.SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Check the solver termination condition
if str(results.solver.termination_condition) == "optimal":
    print("Solver terminated with optimal solution.")
else:
    print("Solver terminated with non-optimal solution.")

Solver terminated with optimal solution.


In [31]:
# Solve the model
solver = pyo.SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
results = solver.solve(model)

# Print the values of decision variables after solving the model
print("Values of decision variables:")
for teacher in model.teachers:
    for day in model.days:
        for time in model.time_slots:
            for lesson in model.lessons:
                print(f"Teacher {teacher}, Day {day}, Time {time}, Lesson {lesson}: {model.schedule[teacher, day, time, lesson].value}")

# Check the solver termination condition
if str(results.solver.termination_condition) == "optimal":
    print("Solver terminated with optimal solution.")
else:
    print("Solver terminated with non-optimal solution.")


Values of decision variables:
Teacher John, Day Monday, Time Morning_09_11, Lesson Math_1: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Math_2: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Science1: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson English1: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson IT: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson IT_LAB: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson History: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Literature: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Art: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Music: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Drama: 0.0
Teacher John, Day Monday, Time Morning_09_11, Lesson Creative_Writing: 0.0
Teacher John, Day Monday, Time Morning_11_13, Lesson Math_1: 0.0
Teacher John, Day Monday, Time Morning_11_13, Lesson Math_2: 0.0
Teacher John, Day Monday, Time Morning_11_13, Less

In [30]:
# Print the schedule
print("Schedule:")
for teacher in model.teachers:
    for day in model.days:
        for time in model.time_slots:
            for lesson in model.lessons:
                if model.schedule[teacher, day, time, lesson].value == 1:
                    print(f"Teacher {teacher} teaches {lesson} on {day} during {time}.")


Schedule:
